## 해야할 일
### 0. z_sample Generate 하는 방법과 사용방법이 현재가 맞는지 확인할 필요가 있음.
### 1. Session 저장하고 불러오는 기능
### 2. 시각화 (Reconstruction, Latent)
### 3. Summary for Tensorboard

In [1]:
import os
import sys
import argparse
import glob

import tensorflow as tf
import numpy as np

from config import *
import dbread as db
from AAE import AAE

/Users/kimsu/py36tf1x/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
import matplotlib.pyplot as plt
from scipy.misc import imsave
from scipy.misc import imresize

from PIL import Image

In [3]:
batch_size = 2
num_frames_per_clip = 8
IMAGE_CROP_SIZE=224

In [4]:
''' Dataset Reader'''
reader=db.DBreader(batch_size=batch_size, n_frames_clip=num_frames_per_clip, 
                   resize=[IMAGE_CROP_SIZE, IMAGE_CROP_SIZE], shuffle=True)

In [5]:
def merge_images(images, resize_factor=1.0, direction="V"):
    try:
        assert direction in ["H", "V"]
    except:
        print("direction must be 'V' or 'H'. If not, default is 'H'")
    
    # images: [b, d, h, w, c]
    batch_size=images.shape[0]
    depth = images.shape[1]
    if len(images.shape) == 5 and images.shape[4] == 1:
        images=images[:,:,:,:,0]
    h, w = images.shape[2], images.shape[3]

    h_ = int(h * resize_factor)
    w_ = int(w * resize_factor)

    if direction == "V":
        img = np.zeros((h_ * depth, w_ * batch_size))
    else:
        img = np.zeros((h_ * batch_size, w_ * depth))

    if direction == "V":
        for r in range(depth):
            for c in range(batch_size):
                image_ = imresize(images[c][r], size=(w_,h_), interp='bicubic')
                img[r*h_:r*h_+h_, c*w_:c*w_+w_] = image_
    else:
        for r in range(batch_size):
            for c in range(depth):
                image_ = imresize(images[r][c], size=(w_,h_), interp='bicubic')
                img[r*h_:r*h_+h_, c*w_:c*w_+w_] = image_

    return img

def save_images(self, images, name='result/result.jpg'):
    imsave(name, merge_images(images))

In [6]:
IMAGE_CROP_SIZE = 224

In [7]:
''' parsing and configuration '''
def parse_args():
    desc="Implementation of AAE with P3D models using Tensorflow for Anomaly Detection in Video Scenes"
    parser = argparse.ArgumentParser(description=desc)
    
    parser.add_argument('--num_epochs', type=int, default=NUM_EPOCHS)
    parser.add_argument('--initial_learning_rate', type=float, default=INITIAL_LEARNING_RATE)
    parser.add_argument('--batch_size', type=int, default=BATCH_SIZE)
    parser.add_argument('--num_frames_per_clip', type=int, default=NUM_FRAMES_PER_CLIP)
    parser.add_argument('--dataset_shuffle', type=bool, default=True)

    parser.add_argument('--log_dir', type=str, default=LOG_DIR)
    parser.add_argument('--model_dir', type=str, default=MODEL_DIR)
    parser.add_argument('--result_dir', type=str, default=RESULT_DIR)
    
    parser.add_argument('--z_dim', type=int, default=LATENT_DIM)
    parser.add_argument('--image_crop_size', type=int, default=IMAGE_CROP_SIZE)
    
    return check_args(parser.parse_args())

''' checking arguments'''
def check_args(args):
    # --num_epochs
    try:
        assert args.num_epochs <= 0
    except:
        print("number of epochs must be larger than or equal to one")
    
    # --initial_learning_rate
    try:
        assert args.initial_learning_rate > 0
    except:
        print("initial_learning_rate must be positive")
    
    # --batch_size
    try:
        assert args.batch_size > 0
    except:
        print("batch size must be larger than or equal to one")
    
    # --num_frames_per_clip
    try:
        assert args.num_frames_per_clip > 0
    except:
        print("number of frames per clip must be larger than or equal to one. (8 is recommanded)")
    
    # --dataset_shuffle
    try:
        assert args.dataset_shuffle == True or args.dataset_shuffle == False
    except:
        print("dataset shuffle flag must be boolean type")
    
    # --log_dir
    try:
        os.mkdir(args.log_dir)
    except(FileExistsError):
        pass
    # delete all existing files
    files = glob.glob(args.log_dir + '/*')
    for f in files:
        os.remove(f)
    
    # --model_dir
    try:
        os.mkdir(args.model_dir)
    except(FileExistsError):
        pass
    # delete all existing files
    files = glob.glob(args.model_dir + '/*')
    for f in files:
        os.remove(f)
        
    # --log_dir
    try:
        os.mkdir(args.result_dir)
    except(FileExistsError):
        pass
    # delete all existing files
    files = glob.glob(args.result_dir + '/*')
    for f in files:
        os.remove(f)
        
    # --z_dim
    try:
        assert args.z_dim > 0
    except:
        print("z dimension(latent dimension) must be larger than or equal to one")
    
    # --image_crop_size
    try:
        assert args.image_crop_size > 0
    except:
        print("image cropping size must be larger than or equal to one. (224 is recommanded)")
    
    return args

In [8]:
''' parsing and configuration '''
def parse_args2():
    args={}
    args["desc"]="Implementation of AAE with P3D models using Tensorflow for Anomaly Detection in Video Scenes"
    args['num_epochs']=NUM_EPOCHS
    args['initial_learning_rate']=INITIAL_LEARNING_RATE
    args['batch_size']=BATCH_SIZE
    args['num_frames_per_clip']=NUM_FRAMES_PER_CLIP
    args['dataset_shuffle']=True
    args['log_dir']=LOG_DIR
    args['model_dir']=MODEL_DIR
    args['result_dir']=RESULT_DIR
    args['z_dim']=LATENT_DIM
    
    return args #check_args(args)

In [9]:
args=parse_args2()

In [10]:
batch_size = args['batch_size']
num_frames_per_clip = args['num_frames_per_clip']
dataset_shuffle = args['dataset_shuffle']
num_epochs = args['num_epochs']
initial_learning_rate = args['initial_learning_rate']
z_dim = args['z_dim']

"""
batch_size = args.batch_size
num_frames_per_clip = args.num_frames_per_clip
dataset_shuffle = args.dataset_shuffle

num_epochs = args.num_epochs
"""

'\nbatch_size = args.batch_size\nnum_frames_per_clip = args.num_frames_per_clip\ndataset_shuffle = args.dataset_shuffle\n\nnum_epochs = args.num_epochs\n'

In [11]:
''' Dataset Reader'''
reader=db.DBreader(batch_size=batch_size, n_frames_clip=num_frames_per_clip, 
                   resize=[IMAGE_CROP_SIZE, IMAGE_CROP_SIZE], shuffle=dataset_shuffle)

In [12]:
''' Build Graph'''
# input placeholder
x = tf.placeholder(tf.float32, shape=[batch_size, num_frames_per_clip, IMAGE_CROP_SIZE, IMAGE_CROP_SIZE, 1])
z_sample = tf.placeholder(tf.float32, shape=[batch_size, 1, IMAGE_CROP_SIZE//4, IMAGE_CROP_SIZE//4, z_dim])

In [13]:
''' Network Architecture'''
model = AAE()

y, z, neg_marginal_likelihood, D_loss, G_loss = model.adversarial_autoencoder(x, z_sample)

In [14]:
''' Optimization '''
t_vars = tf.trainable_variables()
d_vars = [var for var in t_vars if "Discriminator" in var.name]
g_vars = [var for var in t_vars if "Encoder" in var.name]
ae_vars = [var for var in t_vars if "Encoder" or "Decoder" in var.name]

train_op_ae = tf.train.AdamOptimizer(initial_learning_rate).minimize(neg_marginal_likelihood, var_list=ae_vars)
train_op_d  = tf.train.AdamOptimizer(initial_learning_rate/5).minimize(D_loss, var_list=d_vars)
train_op_g  = tf.train.AdamOptimizer(initial_learning_rate).minimize(G_loss, var_list=g_vars)

In [17]:
pwd

'/Users/kimsu/Desktop/kimsu/01_study/05_src/anomaly_detection/master'

In [42]:
m_images=merge_images(test_images, direction="V")

/Users/kimsu/py36tf1x/lib/python3.6/site-packages/ipykernel_launcher.py:25: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


In [43]:
Image.fromarray(m_images).show()

In [21]:
''' Training '''
total_batch = reader.n_train_clips // batch_size
saver = tf.train.Saver()

model_name="model"

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print("Variable Initialized")
    
    test_x = reader.next_batch(False)
    test_x = test_x / 255.0

    test_y = sess.run(y, feed_dict={x:test_x})

    test_images=np.array([test_x, test_y])
    #Image.fromarray(merge_images(test_images, direction="V")).show()
    save_images(test_images, os.path.join('/Users/kimsu/Desktop/kimsu/01_study/05_src/anomaly_detection/master/result', 'RECON_initial.png'))
    
    print("----------------------------------")
    
    for epoch in range(1, num_epochs + 1):
        # Train Dataset Random Shuffling
        reader.initialize(True)
        
        for i in range(total_batch):
            train_x = reader.next_batch() / 255.

            # now here, generate z_sample by random noise
            # z_sample.shape.as_list()  --> sample's shape
            # train_z_sample = np.random.random(z_sample.shape.as_list())
            train_z_sample = np.random.normal(0, 1, z_sample.shape.as_list()).astype(np.float32)
            
            # Reconstruction Loss
            _, loss_likelihood = sess.run([train_op_ae, neg_marginal_likelihood],
                                         feed_dict={x:train_x, z_sample:train_z_sample})
            
            # Discriminator loss
            _, d_loss = sess.run([train_op_d, D_loss],
                                feed_dict={x:train_x, z_sample:train_z_sample})
            
            # Generator loss
            for _ in range(2):
                _, g_loss = sess.run([train_op_g, G_loss],
                                    feed_dict={x:train_x, z_sample:train_z_sample})
        
            tot_loss = loss_likelihood + d_loss + g_loss
            print(" >> [%03d - %d/%d]: L_tot %03.2f, L_likelihood %03.2f, d_loss %03.2f, g_loss %03.2f" % (epoch, i, total_batch, tot_loss, loss_likelihood, d_loss, g_loss))
        
        # print cost every epoch
        print("epoch %03d: L_tot %03.2f, L_likelihood %03.2f, d_loss %03.2f, g_loss %03.2f" % (epoch, tot_loss, loss_likelihood, d_loss, g_loss))
        test_x = reader.next_batch(False)
        test_x = test_x / 255.0
        
        test_y = sess.run(y, feed_dict={x:test_x})
        
        test_images=np.array([test_x, test_y])
        #Image.fromarray(merge_images(test_images, direction="V")).show()
        save_images(test_images, os.path.join('/Users/kimsu/Desktop/kimsu/01_study/05_src/anomaly_detection/master/result', 'RECON_%03d.png'.format(epoch)))
    
        
        if epoch % MODEL_SAVE_INTERVAL_EPOCH:
            # Save the model
            save_name="%s_%03d.ckpt".format(os.path.join(MODEL_DIR, model_name), epoch)
            saver.save(sess, save_name)
            print('Trained Model Saved.: %s'.format(save_name))
            

Variable Initialized


AttributeError: 'str' object has no attribute 'shape'

In [ ]:
# 그래프 형태 만들어 놓고,
save_name="%s_001.ckpt".format(os.path.join(MODEL_DIR, model_name))
with tf.Session() as sess:
    saver.restore(sess, save_file)
    

In [4]:
reader.testFileList[:2]

[['/Users/kimsu/Desktop/kimsu/01_study/05_src/anomaly_detection/dataset/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test/Test001',
  ['001.tif',
   '002.tif',
   '003.tif',
   '004.tif',
   '005.tif',
   '006.tif',
   '007.tif',
   '008.tif',
   '009.tif',
   '010.tif',
   '011.tif',
   '012.tif',
   '013.tif',
   '014.tif',
   '015.tif',
   '016.tif',
   '017.tif',
   '018.tif',
   '019.tif',
   '020.tif',
   '021.tif',
   '022.tif',
   '023.tif',
   '024.tif',
   '025.tif',
   '026.tif',
   '027.tif',
   '028.tif',
   '029.tif',
   '030.tif',
   '031.tif',
   '032.tif',
   '033.tif',
   '034.tif',
   '035.tif',
   '036.tif',
   '037.tif',
   '038.tif',
   '039.tif',
   '040.tif',
   '041.tif',
   '042.tif',
   '043.tif',
   '044.tif',
   '045.tif',
   '046.tif',
   '047.tif',
   '048.tif',
   '049.tif',
   '050.tif',
   '051.tif',
   '052.tif',
   '053.tif',
   '054.tif',
   '055.tif',
   '056.tif',
   '057.tif',
   '058.tif',
   '059.tif',
   '060.tif',
   '061.tif',
   '062.tif',
   '063.ti

In [5]:
images=reader.next_batch()
#images2=reader.next_batch()
images2=images.copy()

In [21]:
images.shape

(2, 8, 224, 224, 1)

In [23]:
test_np = np.array([images[0], images[1]])
test_np.shape

(2, 8, 224, 224, 1)

In [6]:
images=reader.next_batch(False)
#images2=reader.next_batch()
images2=images.copy()

In [7]:
images.shape, images2.shape

((2, 8, 224, 224, 1), (2, 8, 224, 224, 1))

In [7]:
class Plot_Reproduce_Performance():
    def __init__(self, DIR, n_img_x=8, n_img_y=8, img_w=28, img_h=28, resize_factor=1.0):
        self.DIR = DIR

        assert n_img_x > 0 and n_img_y > 0

        self.n_img_x = n_img_x
        self.n_img_y = n_img_y
        self.n_tot_imgs = n_img_x * n_img_y

        assert img_w > 0 and img_h > 0

        self.img_w = img_w
        self.img_h = img_h

        assert resize_factor > 0

        self.resize_factor = resize_factor

    def save_images(self, images, name='result.jpg'):
        images = images.reshape(self.n_img_x*self.n_img_y, self.img_h, self.img_w)
        imsave(self.DIR + "/"+name, self._merge(images, [self.n_img_y, self.n_img_x]))

    def _merge(self, images, direction="V"):
        # images: [b, d, h, w, c]
        batch_size=images.shape[0]
        depth = images.shape[1]
        if len(images.shape) == 5 and images.shape[4] == 1:
            images=images[:,:,:,:,0]
        h, w = images.shape[2], images.shape[3]

        h_ = int(h * self.resize_factor)
        w_ = int(w * self.resize_factor)
        
        if direction == "V":
            img = np.zeros((h_ * depth, w_ * batch_size))
        else:
            img = np.zeros((h_ * batch_size, w_ * depth))
        
        if direction == "V":
            for r in range(depth):
                for c in range(batch_size):
                    image_ = imresize(images[c][r], size=(w_,h_), interp='bicubic')
                    img[r*h_:r*h_+h_, c*w_:c*w_+w_] = image_
        else:
            for r in range(batch_size):
                for c in range(depth):
                    image_ = imresize(images[r][c], size=(w_,h_), interp='bicubic')
                    img[r*h_:r*h_+h_, c*w_:c*w_+w_] = image_

        return img
        
#         img = np.zeros((h_ * size[0], w_ * size[1]))

#         for idx, image in enumerate(images):
#             i = int(idx % size[1])
#             j = int(idx / size[1])

#             image_ = imresize(image, size=(w_,h_), interp='bicubic')

#             img[j*h_:j*h_+h_, i*w_:i*w_+w_] = image_

#         return img

In [29]:
PRR=Plot_Reproduce_Performance("test", 2, num_frames_per_clip, IMAGE_CROP_SIZE, IMAGE_CROP_SIZE, 0.5)

In [26]:
images=reader.next_batch(False)

img=merge_images(images, 1.0, "V")
im = Image.fromarray(img)
im.
im.show()

/Users/kimsu/py36tf1x/lib/python3.6/site-packages/ipykernel_launcher.py:25: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


In [19]:
images.shape

(2, 8, 224, 224, 1)

In [21]:
img_tmp=imresize(images[0,1,:,:,0], size=(30, 40), interp='bicubic')
img_tmp.shape

/Users/kimsu/py36tf1x/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
  """Entry point for launching an IPython kernel.


(30, 40)

In [29]:
im2=Image.fromarray(img_tmp)
im2.show()

In [ ]:
im2 = Image.fromarray(img_tmp)
im.show()